# Deep Convolutional Generative Adversarial Network

## Import dependencies

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.__version__

'1.3.0'

In [3]:
np.__version__

'1.13.3'

## Load in data

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [5]:
print('Training:   {:,}'.format(data.train.num_examples))
print('Testing:    {:,}'.format(data.test.num_examples))
print('Validation:  {:,}'.format(data.validation.num_examples))

Training:   55,000
Testing:    10,000
Validation:  5,000


## Define hyperparameters

In [6]:
# Inputs
image_size = 28
image_channel = 1
image_shape = (image_size, image_size, image_channel)
image_size_flat = image_size * image_size * image_channel
num_classes = 10

# Network
filter_size = 5
hidden1_filter = 32
hidden2_filter = 64
fc1_size = 1024
fc2_size = 1
dropout = 0.8


## Helper functions

### `weights` and `biases`

In [7]:
def weight(shape, name):
#     initial = tf.truncated_normal(shape=shape, mean=0, stddev=0.02)
#     return tf.Variable(initial, name='weight')
    return tf.get_variable(name, [5, 5, 1, 32], initializer=tf.truncated_normal_initializer(stddev=0.02))


def bias(shape, name):
#     initial = tf.zeros(shape=[shape])
#     return tf.Variable(initial, name='bias')
    return tf.get_variable(name, [32], initializer=tf.constant_initializer(0))

### `convolution` and `pooling` 

In [8]:
def conv2d(X, W, strides=[1, 1, 1, 1]):
    return tf.nn.conv2d(X, W, strides=strides, padding='SAME')


def max_pool(X):
    return tf.nn.max_pool(X, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### `flatten` layer

In [9]:
def flatten(layer):
    layer_shape = layer.get_shape()
    num_features = np.array(layer_shape[1:4], dtype=int).prod()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, num_features

### Layers

In [10]:
# convolutional layer
def conv_layer(prev_layer, prev_filter, layer_filter, layer_name, 
               strides=[1, 1, 1, 1], filter_size=5, use_pool=True, batch_norm=False):
#     with tf.name_scope(layer_name):
    W = weight(shape=[filter_size, filter_size, prev_filter, layer_filter], name=layer_name+'_W')
    b = bias(shape=layer_filter, name=layer_name + '_b')
    layer = conv2d(prev_layer, W, strides=strides) + b
    if batch_norm:
        layer = tf.contrib.layers.batch_norm(layer, epsilon=1e-3)
    if use_pool:
        layer = max_pool(layer)
    layer = tf.nn.relu(layer)
    return layer


# fully connected layer
def fc_layer(prev_layer, prev_size, layer_size, layer_name, 
             use_relu=True, dropout=False, batch_norm=False):
#     with tf.name_scope(layer_name):
    W = weight(shape=[prev_size, layer_size], name=layer_name + '_W')
    b = bias(shape=layer_size, name=layer_name + '_b')
    layer = tf.matmul(prev_layer, W) + b
    # Use batch normalization
    if batch_norm:
        layer = tf.contrib.layers.batch_norm(layer, epsilon=1e-5)
    # Use ReLU
    if use_relu:
        layer = tf.nn.relu(layer)
    # Add dropout
    if dropout:
        layer = tf.nn.dropout(layer, keep_prob)
    return layer


# Output layers
def output_layer(fc_layer, fc_size, num_classes):
#     with tf.name_scope('output_layer'):
    W = weight(shape=[fc_size, num_classes], name='output_layer_W')
    b = bias(shape=num_classes, name='output_layer_b')
    logits = tf.matmul(fc_layer, W) + b
    y_pred = tf.nn.softmax(logits)
    y_pred_true = tf.argmax(y_pred, axis=1)
    return logits, y_pred_true

### Discrimitor (Convolutional NN)

In [11]:
def discriminator(X_image, reuse=False):
    if reuse:
        tf.get_variable_scope().reuse_variables()
    
    # Convolutional blocks
    hidden1 = conv_layer(X_image, image_channel, hidden1_filter, 'dis_hidden1', use_pool=True)
    hidden2 = conv_layer(hidden1, hidden1_filter, hidden2_filter, 'dis_hidden2', use_pool=True)
    
    # Fully connected blocks
    hidden2_flat, hidden2_flat_filters = flatten(hidden2)
    fc1_layer = fc_layer(hidden2_flat, hidden2_flat_filters, fc1_size, 'dis_fc1', use_relu=True)
    fc2_layer = fc_layer(fc1_layer, fc1_size, fc2_size, 'dis_fc2', use_relu=False)
    
    return fc2_layer

### Generator (Deconvolutional NN)

In [12]:
def generator(batch_size, z_dim, up_scale=56):
    z = tf.truncated_normal(shape=[batch_size, z_dim], mean=0, stddev=1, name='z')
    scale = up_scale * up_scale
    
    # Fully connected block
    fc1_layer = fc_layer(z, z_dim, scale, 'gen_fc1', use_relu=True, batch_norm=True)
    fc1_layer = tf.reshape(fc1_layer, [-1, up_scale, up_scale, image_channel])
    
    # Convolutional block
    hidden1 = conv_layer(fc1_layer, image_channel, z_dim/2, 'gen_deconv1', 
                         strides=[1, 2, 2, 1], filter_size=3, use_pool=False, batch_norm=True)
    hidden1 = tf.image.resize_images(hidden1, size=[up_scale, up_scale])
    
    hidden2 = conv_layer(hidden1, z_dim/2, z_dim/4, 'gen_deconv2', 
                         strides=[1, 2, 2, 1], filter_size=3, use_pool=False, batch_norm=True)
    hidden2 = tf.image.resize_images(hidden2, size=[up_scale, up_scale])
    
    hidden3 = conv_layer(hidden2, z_dim/4, 1, 'gen_deconv3', 
                         strides=[1, 2, 2, 1], filter_size=1, use_pool=False, batch_norm=False)
    hidden3 = tf.nn.sigmoid(hidden3)
    # No batch normalization in the final layer but we add sigmoid activation 
    # to make the generated images more compact/crisper. (batch_size, 28, 28, 1)
    return hidden3

## Building the Network

In [13]:
sess = tf.Session()

batch_size = 50
z_dimensions = 100

X_placeholder = tf.placeholder('float', shape=[None, 28, 28, 1], name='X_placeholder')

Gz = generator(batch_size, z_dimensions)  # the generated image
Dx = discriminator(X_placeholder)   # classify the real image
Dg = discriminator(Gz, reuse=True)  # classify the generated image

ValueError: Shape must be rank 2 but is rank 4 for 'MatMul' (op: 'MatMul') with input shapes: [50,100], [5,5,1,32].

### Loss function
#### generator's loss
The generator wants to optimize the probability that **the generated image is rated highly** i.e _for every image it generates, the discriminator should say it's real_

#### discriminator's loss
The discriminator wants to:
* optimize the probability that **the real data is rated highly**
* optimize the probability that **the generated data is rated poorly**
That is, when it classifies images from the training data, it should say this is the real image and when it classifies images from the generated data, it should say this is the fake image

In [ ]:
# generator's loss
g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Dg, labels=tf.ones_like(Dg)))
# discriminator's loss
d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Dx, labels=tf.ones_like(Dx)))
d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Dg, labels=tf.zeros_like(Dg)))
d_loss = d_loss_real + d_loss_fake